In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pydicom
import cv2

# Define your data directories and parameters
train_data_dir = r'D:\Research\Neurodegenerative Diseases\Images\Task1\Task1-Images\MRI\Training'
test_data_dir = r'D:\Research\Neurodegenerative Diseases\Images\Task1\Task1-Images\MRI\Testing'
batch_size = 32
image_size = (128, 128) 
num_epochs = 2
num_classes = 5  # CN, AD, MCI, EMCI, LMCI

# Function to load and preprocess images and labels
def load_and_preprocess_images_and_labels(directory):
    image_data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".dcm"):
            dcm_path = os.path.join(directory, filename)
            try:
                dcm_data = pydicom.dcmread(dcm_path)
                img = dcm_data.pixel_array.astype(np.float32) / 255.0
                img = cv2.resize(img, image_size)
                image_data.append(img)
                
                # Parse the filename to extract class name
                filename_parts = filename.split('_')
                class_name = filename_parts[-1].split('.')[0]
                
                # Map class names to class labels (you can customize this)
                class_mapping = {'CN': 0, 'AD': 1, 'MCI': 2, 'EMCI': 3, 'LMCI': 4}
                label = class_mapping.get(class_name, -1)  # Assign labels as needed
                
                if label != -1:
                    labels.append(label)
                else:
                    print(f"Unknown class name: {class_name}")
            except Exception as e:
                print(f"Error processing file {dcm_path}: {str(e)}")

    return np.array(image_data), np.array(labels)

# Load and preprocess training and testing data
train_images, train_labels = load_and_preprocess_images_and_labels(train_data_dir)
test_images, test_labels = load_and_preprocess_images_and_labels(test_data_dir)

# Expand grayscale images to three channels (RGB)
train_images = np.repeat(train_images[:, :, :, np.newaxis], 3, axis=-1)
test_images = np.repeat(test_images[:, :, :, np.newaxis], 3, axis=-1)

# Convert labels to one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Rest of your code remains the same...

# Create a data generator for training images
train_data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the ResNet50 model
base_model = ResNet50(weights='imagenet', input_shape=(128, 128, 3), include_top=False)

# Modify the last layer of your model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Use softmax activation

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the ResNet50 model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Apply oversampling only if the class distribution is imbalanced
class_distribution = Counter(np.argmax(train_labels, axis=1))
max_class_samples = max(class_distribution.values())

if len(class_distribution) > 1 and max_class_samples < len(train_labels) / len(class_distribution):
    oversampler = RandomOverSampler(random_state=42)
    train_images_resampled, train_labels_resampled = oversampler.fit_resample(
        train_images.reshape(-1, image_size[0]*image_size[1]*3),
        train_labels.argmax(axis=1)  # Use integer labels
    )

    # Convert integer labels to one-hot encoding
    train_labels_resampled = tf.keras.utils.to_categorical(train_labels_resampled, num_classes)

    # Train the model using the data generator
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        history = model.fit(
            train_data_generator.flow(train_images_resampled, train_labels_resampled, batch_size=batch_size),
            steps_per_epoch=len(train_images_resampled) // batch_size,
            epochs=1,  # Train for one epoch at a time
        )
else:
    # Train the model using the original unbalanced data
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        history = model.fit(
            train_data_generator.flow(train_images, train_labels, batch_size=batch_size),
            steps_per_epoch=len(train_images) // batch_size,
            epochs=1,  # Train for one epoch at a time
        )

# Extract features from the Global Average Pooling layer
feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)
train_features = feature_extractor.predict(train_images)
test_features = feature_extractor.predict(test_images)

# Flatten the extracted features
train_features_flat = train_features.reshape(train_features.shape[0], -1)
test_features_flat = test_features.reshape(test_features.shape[0], -1)

# Apply Logistic Regression classifier
classifier = LogisticRegression(max_iter=1000)
classifier.fit(train_features_flat, train_labels.argmax(axis=1))
test_predictions = classifier.predict(test_features_flat)

# Calculate and print test accuracy
test_accuracy = accuracy_score(np.argmax(test_labels, axis=1), test_predictions)
print(f"Test Accuracy (Logistic Regression): {test_accuracy * 100:.2f}%")

Epoch 1/2
421/421 [==============================] - 582s 1s/step - loss: 1.4721 - accuracy: 0.3085
Epoch 2/2
106/106 [==============================] - 128s 1s/step
Test Accuracy (Logistic Regression): 40.72%


c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
import os
import cv2
import pydicom
import numpy as np
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Function to load, preprocess images, and assign labels
def load_and_preprocess_images_with_labels(directory, image_size):
    image_data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".dcm"):
            dcm_path = os.path.join(directory, filename)
            try:
                dcm_data = pydicom.dcmread(dcm_path)
                img = dcm_data.pixel_array.astype(np.float32) / 255.0
                img = cv2.resize(img, image_size)
                image_data.append(img)
                filename_parts = filename.split('_')
                class_name = filename_parts[-1].split('.')[0]
                class_mapping = {'CN': 0, 'AD': 1, 'MCI': 2, 'EMCI': 3, 'LMCI': 4}
                label = class_mapping.get(class_name, -1)
                if label != -1:
                    labels.append(label)
                else:
                    print(f"Unknown class name: {class_name}")
            except Exception as e:
                print(f"Error processing file {dcm_path}: {str(e)}")

    return np.array(image_data), np.array(labels)

data_dir = r'D:\Alzheimers\Voting2'

# Define image sizes for each model
image_size_fmri = (75, 75)
image_size_mri = (128, 128)
image_size_pet = (128, 128)

test_data_fmri, true_labels = load_and_preprocess_images_with_labels(data_dir, image_size_fmri)
test_data_mri, _ = load_and_preprocess_images_with_labels(data_dir, image_size_mri)
test_data_pet, _ = load_and_preprocess_images_with_labels(data_dir, image_size_pet)

model_fmri = load_model('D:/Alzheimers/Models/fMRI_Task1.h5')
model_mri = load_model('D:/Alzheimers/Models/MRI_Task1.h5')
model_pet = load_model('D:/Alzheimers/Models/PET_Task1.h5')


X_fmri_features = model_fmri.predict(np.repeat(test_data_fmri[:, :, :, np.newaxis], 3, axis=-1)).reshape(len(test_data_fmri), -1)
X_mri_features = model_mri.predict(np.repeat(test_data_mri[:, :, :, np.newaxis], 3, axis=-1)).reshape(len(test_data_mri), -1)
X_pet_features = model_pet.predict(np.repeat(test_data_pet[:, :, :, np.newaxis], 3, axis=-1)).reshape(len(test_data_pet), -1)


# Stack the features into a single array
X_combined = np.hstack((X_fmri_features, X_mri_features, X_pet_features))

# Use labels from the preprocessing step as true_labels
true_labels = true_labels

# Initialize the classifiers
logistic_reg = LogisticRegression(max_iter=1000)
svm_classifier = SVC()
random_forest = RandomForestClassifier()
gradient_boosting = GradientBoostingClassifier()

# Train the classifiers on the combined features
logistic_reg.fit(X_combined, true_labels)
svm_classifier.fit(X_combined, true_labels)
random_forest.fit(X_combined, true_labels)
gradient_boosting.fit(X_combined, true_labels)

# Make predictions with the trained classifiers
y_pred_logistic = logistic_reg.predict(X_combined)
y_pred_svm = svm_classifier.predict(X_combined)
y_pred_rf = random_forest.predict(X_combined)
y_pred_gb = gradient_boosting.predict(X_combined)

# Calculate and print accuracy for each classifier
accuracy_logistic = accuracy_score(true_labels, y_pred_logistic)
accuracy_svm = accuracy_score(true_labels, y_pred_svm)
accuracy_rf = accuracy_score(true_labels, y_pred_rf)
accuracy_gb = accuracy_score(true_labels, y_pred_gb)

print(f"Logistic Regression Accuracy: {accuracy_logistic * 100:.2f}%")
print(f"SVM Accuracy: {accuracy_svm * 100:.2f}%")
print(f"Random Forest Accuracy: {accuracy_rf * 100:.2f}%")
print(f"Gradient Boosting Accuracy: {accuracy_gb * 100:.2f}%")


1563/1563 [==============================] - 715s 455ms/step
Logistic Regression Accuracy: 76.64%
SVM Accuracy: 84.00%
Random Forest Accuracy: 99.94%
Gradient Boosting Accuracy: 93.65%


In [4]:
print("FMRI Model Layers:")
for layer in model_fmri.layers:
    print(layer.name)

print("MRI Model Layers:")
for layer in model_mri.layers:
    print(layer.name)

print("PET Model Layers:")
for layer in model_pet.layers:
    print(layer.name)


FMRI Model Layers:
input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool
global_average_pooling2d
dense
dropout
dense_1
MRI Model Layers:
input_1
rescaling
normalization
rescaling_1
stem_conv_pad
stem_conv
stem_bn
stem_activation
block1a_dwconv
block1a_bn
block1a_activation
block1a_se_squeeze
block1a_se_reshape
block1a_se_reduce
block1a_se_expand
block1a_se_excite
block1a_project_conv
block1a_project_bn
block1b_dwconv
block1b_bn
block1b_activation
block1b_se_squeeze
block1b_se_reshape
block1b_se_reduce
block1b_se_expand
block1b_se_excite
block1b_project_conv
block1b_project_bn
block1b_drop
block1b_add
block2a_expand_conv
block2a_expand_bn
block2a_expand_activation
block2a_dwconv_pad
block2a_dwconv
block2a_bn
block2a_activation
block2a_se_squeeze
block2a_se_reshape
block2a_se_reduce
block2a_se_expand
block2